In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
import keras
from keras import backend as K
print(tf.__version__)
print(keras.__version__)

from tensorflow.keras import optimizers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras import regularizers
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization

In [ ]:
data_test = np.load('test.npy', allow_pickle=True)
data_train = np.load('train.npy', allow_pickle=True)

In [ ]:
data_train = np.concatenate([data1, data2, data3, data4])

In [ ]:
x_train = data_train[:,0]
y_train = data_train[:,1]

x_test = data_test

In [ ]:
# Interpolation 
import cv2

for i in range(x_train.shape[0]):
  x_train[i] = cv2.resize(x_train[i], (77,77), interpolation=cv2.INTER_CUBIC)

In [ ]:
x_train_new = []
for x in x_train:
  x_train_new.append(x)
x_train_new = np.array(x_train_new)
x_train_new = x_train_new.reshape((x_train_new.shape[0], x_train_new.shape[1], x_train_new.shape[2], 1))
print(x_train_new.shape)

In [ ]:
x_train_new.shape[0]

In [ ]:
y_train = np.vstack([y_train, y_train])

In [ ]:
train_gen = ImageDataGenerator(zoom_range=[0.8, 1.2], width_shift_range=0.1, height_shift_range=0.1)
iterator = train_gen.flow(x_train_new, y_train)

In [ ]:
def reset_tf_session():
    curr_session = tf.get_default_session()
    if curr_session is not None:
        curr_session.close()
    tf.keras.backend.clear_session()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    s = tf.InteractiveSession(config=config)
    tf.keras.backend.set_session(s)
    return s

s = reset_tf_session()

In [ ]:
weight_decay = 1e-4
initializer='glorot_normal'

model = Sequential()

model.add(Conv2D(filters=32, padding='same', kernel_size=(11,11), input_shape=(77,77,1), kernel_initializer=initializer))
model.add(LeakyReLU(0.1))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, padding='same', kernel_size=(11,11), kernel_initializer=initializer))
model.add(LeakyReLU(0.1))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
    
model.add(Conv2D(filters=64, padding='same', kernel_size=(11,11), kernel_initializer=initializer))
model.add(LeakyReLU(0.1))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, padding='same', kernel_size=(11,11), kernel_initializer=initializer))
model.add(LeakyReLU(0.1))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(filters=64, padding='same', kernel_size=(7,7), kernel_initializer=initializer))
model.add(LeakyReLU(0.1))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, padding='same', kernel_size=(7,7), kernel_initializer=initializer))
model.add(LeakyReLU(0.1))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
    
model.add(Conv2D(filters=128, padding='same', kernel_size=(3,3), kernel_initializer=initializer))
model.add(LeakyReLU(0.1))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, padding='same', kernel_size=(3,3), kernel_initializer=initializer))
model.add(LeakyReLU(0.1))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(256, kernel_initializer=initializer))
model.add(LeakyReLU(0.1))
model.add(Dense(1000, kernel_initializer=initializer))
model.add(Activation("softmax"))

In [ ]:
INIT_LR = 5e-3

model.compile(
    loss='sparse_categorical_crossentropy',  
    optimizer=keras.optimizers.adamax(lr=INIT_LR),  
    metrics=['accuracy']
)


In [ ]:
model.summary()

In [ ]:
history1 = model.fit_generator(
    iterator,
    steps_per_epoch=4000,
    epochs=30,
    shuffle=True,
    verbose=1,
    initial_epoch=0
) 

In [ ]:
# Interpolation 
import cv2

for i in range(x_test.shape[0]):
  x_test[i] = cv2.resize(x_test[i], (77,77), interpolation=cv2.INTER_CUBIC)

In [ ]:
x_test_new = []
for x in x_test:
  x_test_new.append(x)
x_test_new = np.array(x_test_new)
x_test_new = x_test_new.reshape((x_test_new.shape[0], x_test_new.shape[1], x_test_new.shape[2], 1))
print(x_test_new.shape)

In [ ]:
ans = model.predict_classes(x_test_new, 4000)

In [ ]:
my_submit = pd.DataFrame(data=ans, columns = ['Category'])
my_submit['Id'] = range(1, x_test.shape[0] + 1)

In [ ]:
my_submit.head()

In [ ]:
my_submit.to_csv('submission.csv', index=False)